In [13]:
from statistics import mean
import torch as t
from torch import nn
import torch.nn.functional as F
# from Params import args
from Utils.Utils import pairPredict, contrastLoss
import argparse







In [23]:
import torch as t
import Utils.TimeLogger as logger
# from Utils.TimeLogger import log
# from Params import args
# from Model import Model
# from DataHandler import DataHandler
import numpy as np
import pickle
# from Utils.Utils import *
import os
import setproctitle

In [15]:
def ParseArgs():
	parser = argparse.ArgumentParser(description='Model Params')
	parser.add_argument('--lr', default=1e-3, type=float, help='learning rate')
	parser.add_argument('--batch', default=4096, type=int, help='batch size')
	parser.add_argument('--leaky', default=0.5, type=float, help='slope of leaky relu')
	parser.add_argument('--tstBat', default=256, type=int, help='number of users in a testing batch')
	parser.add_argument('--reg', default=1e-7, type=float, help='weight decay regularizer')
	parser.add_argument('--epoch', default=100, type=int, help='number of epochs')
	parser.add_argument('--decay', default=0.96, type=float, help='weight decay rate')
	parser.add_argument('--save_path', default='tem', help='file name to save model and training record')
	parser.add_argument('--latdim', default=32, type=int, help='embedding size')
	parser.add_argument('--hyperNum', default=12800, type=int, help='number of hyperedges')
	parser.add_argument('--gnn_layer', default=2, type=int, help='number of gnn layers')
	parser.add_argument('--load_model', default=None, help='model name to load')
	parser.add_argument('--topk', default=20, type=int, help='K of top K')
	parser.add_argument('--keepRate', default=0.5, type=float, help='ratio of edges to keep')
	parser.add_argument('--temp', default=1, type=float, help='temperature')
	parser.add_argument('--mult', default=1, type=float, help='multiplication factor')
	parser.add_argument('--ssl_reg', default=1e-3, type=float, help='weight for ssl loss')
	parser.add_argument('--data', default='yelp', type=str, help='name of dataset')
	parser.add_argument('--tstEpoch', default=3, type=int, help='number of epoch to test while training')
	parser.add_argument('--gpu', default='2', type=str, help='indicates which gpu to use')
	return parser.parse_args()
args = ParseArgs()


usage: ipykernel_launcher.py [-h] [--lr LR] [--batch BATCH] [--leaky LEAKY]
                             [--tstBat TSTBAT] [--reg REG] [--epoch EPOCH]
                             [--decay DECAY] [--save_path SAVE_PATH]
                             [--latdim LATDIM] [--hyperNum HYPERNUM]
                             [--gnn_layer GNN_LAYER] [--load_model LOAD_MODEL]
                             [--topk TOPK] [--keepRate KEEPRATE] [--temp TEMP]
                             [--mult MULT] [--ssl_reg SSL_REG] [--data DATA]
                             [--tstEpoch TSTEPOCH] [--gpu GPU]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/dose/.local/share/jupyter/runtime/kernel-8479ba63-4f75-4b89-bea1-f99a0ee032c6.json


SystemExit: 2

In [17]:
args.gpu

NameError: name 'args' is not defined

In [4]:
init = nn.init.xavier_uniform_
uniformInit = nn.init.uniform


In [16]:
class Model(nn.Module):
	def __init__(self):
		super(Model, self).__init__()

		self.uEmbeds = nn.Parameter(init(t.empty(args.user, args.latdim)))
		self.iEmbeds = nn.Parameter(init(t.empty(args.item, args.latdim)))
		self.gcnLayer = GCNLayer()
		self.hgnnLayer = HGNNLayer()
		self.uHyper = nn.Parameter(init(t.empty(args.latdim, args.hyperNum)))
		self.iHyper = nn.Parameter(init(t.empty(args.latdim, args.hyperNum)))

		self.edgeDropper = SpAdjDropEdge()

	def forward(self, adj, keepRate):
		embeds = t.concat([self.uEmbeds, self.iEmbeds], dim=0)
		lats = [embeds]
		gnnLats = []
		hyperLats = []
		print(self.uHyper)
		print(self.uHyper.type())
		# print(self.iHyper)
		self.uHyper = self.uHyper.to_sparse()
		print(self.uHyper)
		uuHyper = self.uEmbeds @ self.uHyper
		iiHyper = self.iEmbeds @ self.iHyper

		for i in range(args.gnn_layer):
			temEmbeds = self.gcnLayer(self.edgeDropper(adj, keepRate), lats[-1])
			hyperULat = self.hgnnLayer(F.dropout(uuHyper, p=1-keepRate), lats[-1][:args.user])
			hyperILat = self.hgnnLayer(F.dropout(iiHyper, p=1-keepRate), lats[-1][args.user:])
			gnnLats.append(temEmbeds)
			hyperLats.append(t.concat([hyperULat, hyperILat], dim=0))
			lats.append(temEmbeds + hyperLats[-1])
		embeds = sum(lats)
		return embeds, gnnLats, hyperLats

	def calcLosses(self, ancs, poss, negs, adj, keepRate):
		embeds, gcnEmbedsLst, hyperEmbedsLst = self.forward(adj, keepRate)
		uEmbeds, iEmbeds = embeds[:args.user], embeds[args.user:]
		
		ancEmbeds = uEmbeds[ancs]
		posEmbeds = iEmbeds[poss]
		negEmbeds = iEmbeds[negs]
		scoreDiff = pairPredict(ancEmbeds, posEmbeds, negEmbeds)
		bprLoss = - (scoreDiff).sigmoid().log().mean()
		# bprLoss = t.maximum(t.zeros_like(scoreDiff), 1 - scoreDiff).mean() * 40

		sslLoss = 0
		for i in range(args.gnn_layer):
			embeds1 = gcnEmbedsLst[i].detach()
			embeds2 = hyperEmbedsLst[i]
			sslLoss += contrastLoss(embeds1[:args.user], embeds2[:args.user], t.unique(ancs), args.temp) + contrastLoss(embeds1[args.user:], embeds2[args.user:], t.unique(poss), args.temp)
		return bprLoss, sslLoss
	
	def predict(self, adj):
		embeds, _, _ = self.forward(adj, 1.0)
		return embeds[:args.user], embeds[args.user:]

class GCNLayer(nn.Module):
	def __init__(self):
		super(GCNLayer, self).__init__()
		self.act = nn.LeakyReLU(negative_slope=args.leaky)

	def forward(self, adj, embeds):
		return (t.spmm(adj, embeds))

class HGNNLayer(nn.Module):
	def __init__(self):
		super(HGNNLayer, self).__init__()
		self.act = nn.LeakyReLU(negative_slope=args.leaky)
	
	def forward(self, adj, embeds):
		# lat = self.act(adj.T @ embeds)
		# ret = self.act(adj @ lat)
		lat = (adj.T @ embeds)
		ret = (adj @ lat)
		return ret

class SpAdjDropEdge(nn.Module):
	def __init__(self):
		super(SpAdjDropEdge, self).__init__()

	def forward(self, adj, keepRate):
		if keepRate == 1.0:
			return adj
		vals = adj._values()
		idxs = adj._indices()
		edgeNum = vals.size()
		mask = ((t.rand(edgeNum) + keepRate).floor()).type(t.bool)
		newVals = vals[mask] / keepRate
		newIdxs = idxs[:, mask]
		return t.sparse.FloatTensor(newIdxs, newVals, adj.shape)


In [ ]:


class Coach:
	def __init__(self, handler):
		self.handler = handler

		print('USER', args.user, 'ITEM', args.item)
		print('NUM OF INTERACTIONS', self.handler.trnLoader.dataset.__len__())
		self.metrics = dict()
		mets = ['Loss', 'preLoss', 'Recall', 'NDCG']
		for met in mets:
			self.metrics['Train' + met] = list()
			self.metrics['Test' + met] = list()

	def makePrint(self, name, ep, reses, save):
		ret = 'Epoch %d/%d, %s: ' % (ep, args.epoch, name)
		for metric in reses:
			val = reses[metric]
			ret += '%s = %.4f, ' % (metric, val)
			tem = name + metric
			if save and tem in self.metrics:
				self.metrics[tem].append(val)
		ret = ret[:-2] + '  '
		return ret

	def run(self):
		self.prepareModel()
		log('Model Prepared')
		if args.load_model != None:
			self.loadModel()
			stloc = len(self.metrics['TrainLoss']) * args.tstEpoch - (args.tstEpoch - 1)
		else:
			stloc = 0
			log('Model Initialized')
		for ep in range(stloc, args.epoch):
			tstFlag = (ep % args.tstEpoch == 0)
			reses = self.trainEpoch()
			log(self.makePrint('Train', ep, reses, tstFlag))
			if tstFlag:
				reses = self.testEpoch()
				log(self.makePrint('Test', ep, reses, tstFlag))
				self.saveHistory()
			print()
		reses = self.testEpoch()
		log(self.makePrint('Test', args.epoch, reses, True))
		self.saveHistory()

	def prepareModel(self):
		self.model = Model().cuda()
		self.opt = t.optim.Adam(self.model.parameters(), lr=args.lr, weight_decay=0)
	
	def trainEpoch(self):
		trnLoader = self.handler.trnLoader
		trnLoader.dataset.negSampling()
		epLoss, epPreLoss = 0, 0
		steps = trnLoader.dataset.__len__() // args.batch
		for i, tem in enumerate(trnLoader):
			ancs, poss, negs = tem
			ancs = ancs.long().cuda()
			poss = poss.long().cuda()
			negs = negs.long().cuda()

			bprLoss, sslLoss = self.model.calcLosses(ancs, poss, negs, self.handler.torchBiAdj, args.keepRate)
			sslLoss = sslLoss * args.ssl_reg

			regLoss = calcRegLoss(self.model) * args.reg
			loss = bprLoss + regLoss + sslLoss
			epLoss += loss.item()
			epPreLoss += bprLoss.item()
			self.opt.zero_grad()
			loss.backward()
			self.opt.step()
			log('Step %d/%d: loss = %.3f, regLoss = %.3f         ' % (i, steps, loss, regLoss), save=False, oneline=True)
		ret = dict()
		ret['Loss'] = epLoss / steps
		ret['preLoss'] = epPreLoss / steps
		return ret

	def testEpoch(self):
		tstLoader = self.handler.tstLoader
		epLoss, epRecall, epNdcg = [0] * 3
		i = 0
		num = tstLoader.dataset.__len__()
		steps = num // args.tstBat
		for usr, trnMask in tstLoader:
			i += 1
			usr = usr.long().cuda()
			trnMask = trnMask.cuda()
			usrEmbeds, itmEmbeds = self.model.predict(self.handler.torchBiAdj)

			allPreds = t.mm(usrEmbeds[usr], t.transpose(itmEmbeds, 1, 0)) * (1 - trnMask) - trnMask * 1e8
			_, topLocs = t.topk(allPreds, args.topk)
			recall, ndcg = self.calcRes(topLocs.cpu().numpy(), self.handler.tstLoader.dataset.tstLocs, usr)
			epRecall += recall
			epNdcg += ndcg
			log('Steps %d/%d: recall = %.2f, ndcg = %.2f          ' % (i, steps, recall, ndcg), save=False, oneline=True)
		ret = dict()
		ret['Recall'] = epRecall / num
		ret['NDCG'] = epNdcg / num
		return ret

	def calcRes(self, topLocs, tstLocs, batIds):
		assert topLocs.shape[0] == len(batIds)
		allRecall = allNdcg = 0
		recallBig = 0
		ndcgBig =0
		for i in range(len(batIds)):
			temTopLocs = list(topLocs[i])
			temTstLocs = tstLocs[batIds[i]]
			tstNum = len(temTstLocs)
			maxDcg = np.sum([np.reciprocal(np.log2(loc + 2)) for loc in range(min(tstNum, args.topk))])
			recall = dcg = 0
			for val in temTstLocs:
				if val in temTopLocs:
					recall += 1
					dcg += np.reciprocal(np.log2(temTopLocs.index(val) + 2))
			recall = recall / tstNum
			ndcg = dcg / maxDcg
			allRecall += recall
			allNdcg += ndcg
		return allRecall, allNdcg

	def saveHistory(self):
		if args.epoch == 0:
			return
		with open('../History/' + args.save_path + '.his', 'wb') as fs:
			pickle.dump(self.metrics, fs)

		content = {
			'model': self.model,
		}
		t.save(content, '../Models/' + args.save_path + '.mod')
		log('Model Saved: %s' % args.save_path)

	def loadModel(self):
		ckp = t.load('../Models/' + args.load_model + '.mod')
		self.model = ckp['model']
		self.opt = t.optim.Adam(self.model.parameters(), lr=args.lr, weight_decay=0)

		with open('../History/' + args.load_model + '.his', 'rb') as fs:
			self.metrics = pickle.load(fs)
		log('Model Loaded')	

if __name__ == '__main__':
	setproctitle.setproctitle('akaxia')
	os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu
	logger.saveDefault = True
	
	log('Start')
	handler = DataHandler()
	handler.LoadData()
	log('Load Data')

	coach = Coach(handler)
	coach.run()

In [26]:
import pickle

# ファイルパス
file_path = 'Data/yelp/trnMat.pkl'

# pickleでファイルを読み込む
with open(file_path, 'rb') as f:
    data = pickle.load(f)

# 読み込んだデータを表示
print(data)


FileNotFoundError: [Errno 2] No such file or directory: 'Data/yelp/trnMat.pkl'

In [27]:
# 元のデータ
data = [
    0.04315965676908131, 0.0932261721434235, 0.1057679653950122, 0.1126046523339154,
    0.11520073598839825, 0.11753018461980097, 0.11874147888303369, 0.11924505180283497,
    0.12033010439201325, 0.12004611942284257, 0.12162617470667342, 0.1208368244130385,
    0.12218977500956586, 0.12132676098885842, 0.12045597182733792, 0.12204749041075832,
    0.12250174352677239, 0.12176249833486022, 0.12099581563196081, 0.12041117806536925,
    0.12213159121415988, 0.12065888419122683, 0.12073688432461452, 0.12126227129029284,
    0.11971658800321978, 0.12177833541643861, 0.12120124270921094, 0.12277635320148475,
    0.12193683380538452, 0.12052066387637578, 0.12184766455755616, 0.12187686252279138,
    0.12171122402944051, 0.12213793191377513, 0.12213793191377513
]

# 有効数字4桁で四捨五入して新しいリストを作成
rounded_data = [round(val, 4) for val in data]

# 結果の表示
print(rounded_data)


[0.0432, 0.0932, 0.1058, 0.1126, 0.1152, 0.1175, 0.1187, 0.1192, 0.1203, 0.12, 0.1216, 0.1208, 0.1222, 0.1213, 0.1205, 0.122, 0.1225, 0.1218, 0.121, 0.1204, 0.1221, 0.1207, 0.1207, 0.1213, 0.1197, 0.1218, 0.1212, 0.1228, 0.1219, 0.1205, 0.1218, 0.1219, 0.1217, 0.1221, 0.1221]


In [35]:
import pandas as pd

# CSVファイルのパス
csv_path = '../Result/tem2024-01-29 17:31:58.csv'

# CSVファイルを読み込む
df = pd.read_csv(csv_path, delimiter=',')

# 'TestRecall'と'TestNDCG'列の右側の値を有効数字4桁で四捨五入
df['TestRecall'] = df['TestRecall'].apply(lambda x: round(x, 4))
df['TestNDCG'] = df['TestNDCG'].apply(lambda x: round(x, 4))

# 結果の表示
print(df)

# 'TestRecall'と'TestNDCG'列をリストに変換
test_recall_list = df['TestRecall'].tolist()
test_ndcg_list = df['TestNDCG'].tolist()

# 結果の表示
print("TestRecall List:", test_recall_list)
print("TestNDCG List:", test_ndcg_list)


    Epoch  TestRecall  TestNDCG
0     100      0.0432    0.0304
1     100      0.0932    0.0665
2     100      0.1058    0.0745
3     100      0.1126    0.0795
4     100      0.1152    0.0814
5     100      0.1175    0.0829
6     100      0.1187    0.0844
7     100      0.1192    0.0847
8     100      0.1203    0.0852
9     100      0.1200    0.0855
10    100      0.1216    0.0862
11    100      0.1208    0.0859
12    100      0.1222    0.0860
13    100      0.1213    0.0862
14    100      0.1205    0.0856
15    100      0.1220    0.0865
16    100      0.1225    0.0864
17    100      0.1218    0.0861
18    100      0.1210    0.0857
19    100      0.1204    0.0848
20    100      0.1221    0.0866
21    100      0.1207    0.0856
22    100      0.1207    0.0859
23    100      0.1213    0.0861
24    100      0.1197    0.0852
25    100      0.1218    0.0861
26    100      0.1212    0.0858
27    100      0.1228    0.0869
28    100      0.1219    0.0866
29    100      0.1205    0.0853
30    10

In [31]:
import pandas as pd

# CSVファイルのパス
csv_path = '../Result/tem2024-01-29 17:31:58.csv'

# CSVファイルを読み込む
df = pd.read_csv(csv_path, delimiter=',')

In [32]:
df

,Epoch,TestRecall,TestNDCG
0,100,0.043160,0.030396
1,100,0.093226,0.066504
2,100,0.105768,0.074496
3,100,0.112605,0.079522
4,100,0.115201,0.081400
5,100,0.117530,0.082888
6,100,0.118741,0.084368
7,100,0.119245,0.084696
8,100,0.120330,0.085201
9,100,0.120046,0.085504


In [ ]:
import pandas as pd

# CSVファイルのパス
csv_path = 'your_file.csv'

# CSVファイルをヘッダーで区切って読み込む
df = pd.read_csv(csv_path, delimiter=',', header=0)

# 'TestRecall'と'TestNDCG'列の右側の値を有効数字4桁で四捨五入
df['TestRecall'] = df['TestRecall'].apply(lambda x: round(x, 4))
df['TestNDCG'] = df['TestNDCG'].apply(lambda x: round(x, 4))

# 'TestRecall'と'TestNDCG'列を配列に変換
test_recall_array = df['TestRecall'].to_numpy()
test_ndcg_array = df['TestNDCG'].to_numpy()

# 結果の表示
print("TestRecall Array:", test_recall_array)
print("TestNDCG Array:", test_ndcg_array)
